# Mining testimonial fragments of the Holocaust

**Experience domain: sit**

### Load the necessary libraries

In [20]:
import sys; sys.path.insert(0, '..')
import itertools

In [21]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [22]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [23]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [24]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [25]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [26]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [27]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [28]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [29]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [30]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Add the main node

In [12]:
main_node = "die"
#delete_main_node("numbness")
add_main_node(main_node)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


### Set up the query

In [31]:
query = '[lemma="die"]'

In [32]:
result = topic_concordancer.main(query,window=50,topicn=25)

Training lda model began
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5Blemma%3D%22die%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=50


### Print the key topics

In [33]:
for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

0
0.044*"story" + 0.039*"talk" + 0.039*"talk_about" + 0.028*"speak" + 0.025*"change" + 0.024*"let_'s" + 0.023*"learn" + 0.021*"reason" + 0.021*"Holocaust" + 0.020*"very_much"


1
0.083*"friend" + 0.038*"ago" + 0.036*"school" + 0.035*"meet" + 0.026*"book" + 0.026*"gon_na" + 0.024*"love" + 0.019*"very_good" + 0.019*"America" + 0.018*"wonderful"


2
0.082*"jewish" + 0.064*"Jews" + 0.058*"german" + 0.034*"Poland" + 0.029*"fight" + 0.026*"jew" + 0.024*"city" + 0.023*"polish" + 0.018*"Germans" + 0.018*"Hitler"


3
0.137*"day" + 0.083*"Auschwitz" + 0.059*"train" + 0.029*"how_many" + 0.028*"arrive" + 0.027*"number" + 0.022*"transport" + 0.018*"wagon" + 0.018*"point" + 0.018*"Theresienstadt"


4
0.146*"camp" + 0.043*"people" + 0.035*"typhus" + 0.028*"many_people" + 0.028*"prisoner" + 0.025*"march" + 0.025*"most_of" + 0.023*"terrible" + 0.023*"starvation" + 0.023*"liberate"


5
0.108*"life" + 0.059*"thing" + 0.058*"good" + 0.057*"feel" + 0.042*"fact" + 0.037*"save" + 0.033*"understand" + 0.030*"

### Analyze documents

In [ ]:
i=0
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

## Testimonial fragments

### 1.  

In [111]:
lemmas = []

In [112]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

([lemma="kill"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="kill"])


In [113]:
domain_term = ""

In [114]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [115]:
fragment_1 = {}
fragment_1['original_sentence'] = ""
fragment_1['label']=""
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%223%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22Then%22%5D+%5B%22they%22%5D+%5B%22said%22%5D+%5B%5D+%5B%22you%22%5D+%5B%5D+%5B%22out%22%5D+%5B%22of%22%5D+%5B%22luck%22%5D+%5B%22because%22%5D+%5B%22we%22%5D+%5B%5D+%5B%22going%22%5D+%5B%22to%22%5D+%5B%22kill%22%5D+%5B%22you%22%5D+%5B%22all%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%224%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22We%22%5D+%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%224%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22were%22%5D+%5B%22numb%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "men . [ PAUSES FOR 3 SECONDS ] Then they said , you 're out of luck because we 're going to kill you all . [ PAUSES FOR 4 SECONDS ] We [ PAUSES FOR 4 SECONDS ] were numb . ", 'right': '', 'complete_match': "men . [ 

In [116]:
fragment_2 = {}
fragment_2['original_sentence'] = ""
fragment_2['label']=""
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22To%22%5D+%5B%22see%22%5D+%5B%22another%22%5D+%5B%5D+%5B%22Jews%22%5D+%5B%22come%22%5D+%5B%22and%22%5D+%5B%22be%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22another%22%5D+%5B%5D+%5B%22be%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22That%22%5D+%5B%22was%22%5D+%5B%22not%22%5D+%5B%5D%7B0%2C50%7D+%5B%22that%22%5D+%5B%22we%22%5D+%5B%22wanted%22%5D+%5B%22to%22%5D+%5B%22see%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%5D+%5B%22Then%22%5D+%5B%22after%22%5D+%5B%22awhile%22%5D+%5B%22you%22%5D+%5B%22become%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%22that%22%5D+%5B%22you%22%5D+%5B%22just%22%5D+%5B%5D%7B0%2C50%7D+%5B%22you%22%5D+%5B%22do%22%5D+%5B%5D+%5B%22react%22%5D+%5B%5D+%5B%22you%22%5D+%5B%5D%7B0%2C50%7D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "To see another 10,000 Jews come and be killed , ano

In [117]:
fragment_3 = {}
fragment_3['original_sentence'] = ""
fragment_3['label']=""
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%5D+%5B%22Cathy%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22every%22%5D+%5B%2210%22%5D+%5B%22people%22%5D+%5B%22get%22%5D+%5B%22killed%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And we were so numb , [ ? Cathy ? ] every 10 people get killed . ', 'right': '', 'complete_match': 'And we were so numb , [ ? Cathy ? ] every 10 people get killed . ', 'testimony_id': 'usc_shoah_25287', 'shelfmark': ['USC 25287'], 'token_start': 16040, 'token_end': 16057}


In [118]:
fragment_4 = {}
fragment_4['original_sentence'] = ""
fragment_4['label']= ""
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22from%22%5D+%5B%22people%22%5D+%5B%22being%22%5D+%5B%22wounded%22%5D+%5B%5D+%5B%22from%22%5D+%5B%22people%22%5D+%5B%22being%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22from%22%5D+%5B%22cries%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22finally%22%5D+%5B%5D+%5B%22everything%22%5D+%5B%22subsided%22%5D+%5B%5D+%5B%22But%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22alive%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22even%22%5D+%5B%22think%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22numb%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "from people being wounded , from people being killed , from cries . And finally , everything subsided . But I was alive . I did n't -- I did n't even think . I was numb . ", 'right': '', 'complete_match': "from people being wounded , from people being killed , from cries . And finally , ev

In [ ]:
fragment_5 = {}
fragment_5['original_sentence'] = ""
fragment_5['label']= ""
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

In [119]:
add_testimonial_fragments(fragments)

### 3.  

In [34]:
lemmas = ["shoot","rape"]

In [35]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

([lemma="shoot"][]{0,50}[lemma="rape"])|([lemma="rape"][]{0,50}[lemma="shoot"])


In [36]:
domain_term = "shoot"

In [114]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [115]:
fragment_1 = {}
fragment_1['original_sentence'] = "They probably raped them, abused them, eventually shot them because nobody survived"
fragment_1['label']="They probably raped them, abused them, eventually shot them because nobody survived"
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%223%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22Then%22%5D+%5B%22they%22%5D+%5B%22said%22%5D+%5B%5D+%5B%22you%22%5D+%5B%5D+%5B%22out%22%5D+%5B%22of%22%5D+%5B%22luck%22%5D+%5B%22because%22%5D+%5B%22we%22%5D+%5B%5D+%5B%22going%22%5D+%5B%22to%22%5D+%5B%22kill%22%5D+%5B%22you%22%5D+%5B%22all%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%224%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22We%22%5D+%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%224%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22were%22%5D+%5B%22numb%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "men . [ PAUSES FOR 3 SECONDS ] Then they said , you 're out of luck because we 're going to kill you all . [ PAUSES FOR 4 SECONDS ] We [ PAUSES FOR 4 SECONDS ] were numb . ", 'right': '', 'complete_match': "men . [ 

In [116]:
fragment_2 = {}
fragment_2['original_sentence'] = "They picked the nice looking girls, they raped them and then they shot them."
fragment_2['label']="They picked the nice looking girls, they raped them and then they shot them."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22To%22%5D+%5B%22see%22%5D+%5B%22another%22%5D+%5B%5D+%5B%22Jews%22%5D+%5B%22come%22%5D+%5B%22and%22%5D+%5B%22be%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22another%22%5D+%5B%5D+%5B%22be%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22That%22%5D+%5B%22was%22%5D+%5B%22not%22%5D+%5B%5D%7B0%2C50%7D+%5B%22that%22%5D+%5B%22we%22%5D+%5B%22wanted%22%5D+%5B%22to%22%5D+%5B%22see%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%5D+%5B%22Then%22%5D+%5B%22after%22%5D+%5B%22awhile%22%5D+%5B%22you%22%5D+%5B%22become%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%22that%22%5D+%5B%22you%22%5D+%5B%22just%22%5D+%5B%5D%7B0%2C50%7D+%5B%22you%22%5D+%5B%22do%22%5D+%5B%5D+%5B%22react%22%5D+%5B%5D+%5B%22you%22%5D+%5B%5D%7B0%2C50%7D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "To see another 10,000 Jews come and be killed , ano

In [117]:
fragment_3 = {}
fragment_3['original_sentence'] = "was raped and shot in the middle of the street."
fragment_3['label']="(..) name was Hannah Prusak, P-R-U-S-A-K-- was raped and shot in the middle of the street."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%5D+%5B%22Cathy%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22every%22%5D+%5B%2210%22%5D+%5B%22people%22%5D+%5B%22get%22%5D+%5B%22killed%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And we were so numb , [ ? Cathy ? ] every 10 people get killed . ', 'right': '', 'complete_match': 'And we were so numb , [ ? Cathy ? ] every 10 people get killed . ', 'testimony_id': 'usc_shoah_25287', 'shelfmark': ['USC 25287'], 'token_start': 16040, 'token_end': 16057}


In [118]:
fragment_4 = {}
fragment_4['original_sentence'] = "I know she was raped. I know that they put a big stick through her genitals. I know that they tortured her before they shot her. "
fragment_4['label']= "I know she was raped. (.. ) I know that they tortured her before they shot her."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22from%22%5D+%5B%22people%22%5D+%5B%22being%22%5D+%5B%22wounded%22%5D+%5B%5D+%5B%22from%22%5D+%5B%22people%22%5D+%5B%22being%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22from%22%5D+%5B%22cries%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22finally%22%5D+%5B%5D+%5B%22everything%22%5D+%5B%22subsided%22%5D+%5B%5D+%5B%22But%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22alive%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22even%22%5D+%5B%22think%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22numb%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "from people being wounded , from people being killed , from cries . And finally , everything subsided . But I was alive . I did n't -- I did n't even think . I was numb . ", 'right': '', 'complete_match': "from people being wounded , from people being killed , from cries . And finally , ev

In [119]:
add_testimonial_fragments(fragments)

### 1.  

In [111]:
lemmas = []

In [112]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

([lemma="kill"][]{0,50}[lemma="numb"])|([lemma="numb"][]{0,50}[lemma="kill"])


In [113]:
domain_term = ""

In [114]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [115]:
fragment_1 = {}
fragment_1['original_sentence'] = ""
fragment_1['label']=""
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%223%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22Then%22%5D+%5B%22they%22%5D+%5B%22said%22%5D+%5B%5D+%5B%22you%22%5D+%5B%5D+%5B%22out%22%5D+%5B%22of%22%5D+%5B%22luck%22%5D+%5B%22because%22%5D+%5B%22we%22%5D+%5B%5D+%5B%22going%22%5D+%5B%22to%22%5D+%5B%22kill%22%5D+%5B%22you%22%5D+%5B%22all%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%224%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22We%22%5D+%5B%5D%7B0%2C3%7D+%5B%22PAUSES%22%5D+%5B%22FOR%22%5D+%5B%224%22%5D+%5B%22SECONDS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22were%22%5D+%5B%22numb%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "men . [ PAUSES FOR 3 SECONDS ] Then they said , you 're out of luck because we 're going to kill you all . [ PAUSES FOR 4 SECONDS ] We [ PAUSES FOR 4 SECONDS ] were numb . ", 'right': '', 'complete_match': "men . [ 

In [116]:
fragment_2 = {}
fragment_2['original_sentence'] = ""
fragment_2['label']=""
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22To%22%5D+%5B%22see%22%5D+%5B%22another%22%5D+%5B%5D+%5B%22Jews%22%5D+%5B%22come%22%5D+%5B%22and%22%5D+%5B%22be%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22another%22%5D+%5B%5D+%5B%22be%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22That%22%5D+%5B%22was%22%5D+%5B%22not%22%5D+%5B%5D%7B0%2C50%7D+%5B%22that%22%5D+%5B%22we%22%5D+%5B%22wanted%22%5D+%5B%22to%22%5D+%5B%22see%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22had%22%5D+%5B%22to%22%5D+%5B%5D+%5B%22Then%22%5D+%5B%22after%22%5D+%5B%22awhile%22%5D+%5B%22you%22%5D+%5B%22become%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%22that%22%5D+%5B%22you%22%5D+%5B%22just%22%5D+%5B%5D%7B0%2C50%7D+%5B%22you%22%5D+%5B%22do%22%5D+%5B%5D+%5B%22react%22%5D+%5B%5D+%5B%22you%22%5D+%5B%5D%7B0%2C50%7D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "To see another 10,000 Jews come and be killed , ano

In [117]:
fragment_3 = {}
fragment_3['original_sentence'] = ""
fragment_3['label']=""
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22so%22%5D+%5B%22numb%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%5D+%5B%22Cathy%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22every%22%5D+%5B%2210%22%5D+%5B%22people%22%5D+%5B%22get%22%5D+%5B%22killed%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And we were so numb , [ ? Cathy ? ] every 10 people get killed . ', 'right': '', 'complete_match': 'And we were so numb , [ ? Cathy ? ] every 10 people get killed . ', 'testimony_id': 'usc_shoah_25287', 'shelfmark': ['USC 25287'], 'token_start': 16040, 'token_end': 16057}


In [118]:
fragment_4 = {}
fragment_4['original_sentence'] = ""
fragment_4['label']= ""
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22from%22%5D+%5B%22people%22%5D+%5B%22being%22%5D+%5B%22wounded%22%5D+%5B%5D+%5B%22from%22%5D+%5B%22people%22%5D+%5B%22being%22%5D+%5B%22killed%22%5D+%5B%5D+%5B%22from%22%5D+%5B%22cries%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22finally%22%5D+%5B%5D+%5B%22everything%22%5D+%5B%22subsided%22%5D+%5B%5D+%5B%22But%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22alive%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22even%22%5D+%5B%22think%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22numb%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "from people being wounded , from people being killed , from cries . And finally , everything subsided . But I was alive . I did n't -- I did n't even think . I was numb . ", 'right': '', 'complete_match': "from people being wounded , from people being killed , from cries . And finally , ev

In [ ]:
fragment_5 = {}
fragment_5['original_sentence'] = ""
fragment_5['label']= ""
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

In [119]:
add_testimonial_fragments(fragments)